In [1]:
import os
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv("openai.env")

# Read the OPENAI_API_KEY from the environment
api_key = os.getenv("OPENAI_API_KEY")
api_base = os.getenv("OPENAI_API_BASE")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_API_BASE"] = api_base

# 在agent与tool之间共享记忆

- 自定义一个工具用来LLMChain来总结内容
- 使用readonlymemory来共享记忆
- 观察共享与不共享的区别
<hr>

In [2]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate,MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper

创建llm

In [3]:
llm = OpenAI(
    temperature = 0,
    model="gpt-3.5-turbo-instruct",
)

/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


创建一条链来总结对话

In [4]:
template = """以下是一段AI机器人和人类的对话:
{chat_history}
根据输入和上面的对话记录写一份对话总结.
输入: {input}"""

prompt = PromptTemplate(
    input_variables=["input","chat_history"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)


构建工具

In [5]:
import os
os.environ["SERPAPI_API_KEY"] = "f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594"
#搜索工具
search = SerpAPIWrapper()
#总结工具
def SummaryChainFun(history):
    print("\n==============总结链开始运行==============")
    print("输入历史: ",history)
    summary_chain.run(history)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="当需要了解实时的信息或者你不知道的事时候可以使用搜索工具",
    ),
    Tool(
        name="Summary",
        func=SummaryChainFun,
        description="当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用",
    ),
]
print(tools)

[Tool(name='Search', description='当需要了解实时的信息或者你不知道的事时候可以使用搜索工具', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='f265b8d9834ed7692cba6db6618e2a8a9b24ed6964c457296a2626026e8ed594', aiosession=None)>), Tool(name='Summary', description='当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用', func=<function SummaryChainFun at 0x11f3537e0>)]


创建记忆组件

In [6]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

创建agent

In [7]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
print(agent_chain.agent.llm_chain.prompt.template)


Answer the following questions as best you can. You have access to the following tools:

Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [9]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"
{chat_history}
Question: {input}
{agent_scratchpad}"""

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix":prefix,
        "suffix":suffix,
        "agent_scratchpad":MessagesPlaceholder("agent_scratchpad"),
        "chat_history":MessagesPlaceholder("chat_history"),
        "input":MessagesPlaceholder("input"),
    },
    memory=memory,
)
print(agent_chain.agent.llm_chain.prompt.template)

Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"
{chat_history}
Question: {input}
{agent_scratchpad}


In [10]:
agent_chain.run(input="美国第45任总统是谁?")



> Entering new AgentExecutor chain...


/Users/tomiezhang/.pyenv/versions/3.10.12/envs/langchains/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Thought: I should use the search tool to find the answer.
Action: Search
Action Input: "Who is the 45th president of the United States?"

ValueError: Got error from SerpAPI: Your account has run out of searches.

In [11]:
agent_chain.run(input="他的夫人叫什么名字?")



> Entering new AgentExecutor chain...
Thought: I should use the search tool to find the answer.
Action: Search
Action Input: "His wife's name"

ValueError: Got error from SerpAPI: Your account has run out of searches.

In [ ]:
print(agent_chain.memory.buffer)

[HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald J. Trump is the 45th President of the United States.'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump')]


In [ ]:
agent_chain.run(input="我们都聊了什么？")



> Entering new AgentExecutor chain...
Thought: I should use the Summary tool to summarize our conversation.
Action: Summary
Action Input: "HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald J. Trump is the 45th President of the United States.'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump')"
==============总结链开始运行==============
输入历史:  HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald J. Trump is the 45th President of the United States.'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump')


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald J. Trump is the 45th President of the United States.'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump')

> Finished chain.

Observation: None
Thought: I now know the final answer
Final Answer: "我们都聊了美国第45任总统是谁，他的夫人叫什么名字。"

> Finished chain.


'"我们都聊了美国第45任总统是谁，他的夫人叫什么名字。"'

In [ ]:
agent_chain.run(input="帮我总结下目前的对话内容，给我5岁的儿子看看")



> Entering new AgentExecutor chain...
Thought: 总结对话内容可以帮助你的儿子更容易理解我们的谈话内容
Action: Summary
Action Input: "我们都聊了美国第45任总统是谁，他的夫人叫什么名字。"
==============总结链开始运行==============
输入历史:  我们都聊了美国第45任总统是谁，他的夫人叫什么名字。


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 我们都聊了美国第45任总统是谁，他的夫人叫什么名字。

> Finished chain.

Observation: None
Thought: 我们的对话内容很简单，可以直接告诉儿子
Action: Summary
Action Input: "我们都聊了美国第45任总统是谁，他的夫人叫什么名字。"
==============总结链开始运行==============
输入历史:  我们都聊了美国第45任总统是谁，他的夫人叫什么名字。


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 我们都聊了美国第45任总统是谁，他的夫人叫什么名字。

> Finished chain.

Observation: None
Thought: 我们的对话内容很简单，可以直接告诉儿子
Action: Summary
Action Input: "我们都聊了美国第45任总统是谁，他的夫人叫什么名字。"
==============总结链开始运行==============
输入历史:  我们都聊了美国第45任总统是谁，他的夫人叫什么名字。


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 我们都聊了美国第45任总统是谁，他的夫人叫什么名字。

> F

'"美国第45任总统是Donald J. Trump，他的夫人叫Melania Trump。"'

In [ ]:
print(agent_chain.memory.buffer)

[HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald J. Trump is the 45th President of the United States.'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump'), HumanMessage(content='我们都聊了什么？'), AIMessage(content='"我们都聊了美国第45任总统是谁，他的夫人叫什么名字。"'), HumanMessage(content='帮我总结下目前的对话内容，给我5岁的儿子看看'), AIMessage(content='"美国第45任总统是Donald J. Trump，他的夫人叫Melania Trump。"')]
